In [1]:
### METODO PARA CONSUMIR EL SERVICIO DE ENTENDIMIENTO DE LENGUAJE NATURAL
import json
from ibm_watson import NaturalLanguageUnderstandingV1
from ibm_watson.natural_language_understanding_v1 import Features, EntitiesOptions,EmotionOptions, KeywordsOptions,MetadataOptions, RelationsOptions, CategoriesOptions, ConceptsOptions, SemanticRolesOptions, SentimentOptions, SyntaxOptions, SyntaxOptionsTokens
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator

authenticator = IAMAuthenticator('mwvcacZF1-9okkmOhNtgvEp7sriD2Ac6PPnUUVaG2A4I')
service = NaturalLanguageUnderstandingV1(
     version='2018-03-16',
     authenticator=authenticator)
service.set_service_url('https://api.us-south.natural-language-understanding.watson.cloud.ibm.com/instances/2d459fe1-9caf-4343-bb54-23da430d22bf')

In [2]:
### METODO PARA CONSUMIR EL SERVICIO DE TRADUCIR 
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator
from ibm_watson import LanguageTranslatorV3

authenticator = IAMAuthenticator('12rCaa3LmgrvGJCVPYwpaXbqi5iY_7ZOscjjPx9wDZ6G')
language_translator = LanguageTranslatorV3(
    version='2018-05-01',
    authenticator=authenticator
)

language_translator.set_service_url('https://api.us-south.language-translator.watson.cloud.ibm.com/instances/9a40898b-8ca6-40cb-9e69-a1dc4ecc3e6f')

In [3]:
from clips import Environment, Symbol
def insertar(res,environment):
    fs1 = '(confirmacionp ' + str(res[0]) + ' )'
    fs2 = '(confirmacionm ' + str(res[1]) + ' )'
    fact1 = environment.assert_string(fs1)
    fact2 = environment.assert_string(fs2)
    
def ejecutar(res):
    environment = Environment()
    environment.load('examen.clp')
    insertar(res,environment)
    environment.run()
    lis=[]
    for fact in environment.facts():
        tt = list(fact)
        for h in tt:
            lis.append(h)
    environment.reset()
    resultado = lis[len(lis)-1]
    if(resultado=="ralto"):
        return "Se recomienda la carrera Administracion de Empresas"
    elif(resultado=="rmedio"):
        return "Se recomienda pero debe estudiar en lo que falla y necesita"
    elif(resultado=="rbajo"):
        return "Si toma la carrera debera estudiar lo maximo posible"

In [4]:
import urllib3
def identLanguage(mensaje):
    language = language_translator.identify(
        mensaje).get_result()
    r= json.dumps(language, indent=2)
    rp= json.loads(r)
    return rp["languages"][0]["language"]

def traducir(mensaje):
    leng=identLanguage(mensaje)
    print("LENguaje Broma ->" ,leng)
    if(leng == "fr" or leng=="ca" or leng=="en"):
        print("Idioma: ",leng)
        translation = language_translator.translate(
        text=mensaje,
        model_id=leng+"-es").get_result()
        r= json.dumps(translation, indent=2)
        rp= json.loads(r)
        return rp["translations"][0]["translation"],leng
    else:
        return mensaje,"es"
    
def traducir2(mensaje,leng):
    print("Lenguaje +",leng)
    if(leng =="fr" or leng=="ca" or leng=="en"):
        print("IAT:",leng)
        translation = language_translator.translate(
        text=mensaje,
        model_id="es-"+leng).get_result()
        r= json.dumps(translation, indent=2)
        rp= json.loads(r)
        return rp["translations"][0]["translation"]
    else:
        return mensaje
    

print(traducir("Hola"))

LENguaje Broma -> es
('Hola', 'es')


In [ ]:
import json
import re
import requests
from ibm_watson import AssistantV2
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator
from ibm_cloud_sdk_core.api_exception import ApiException
import mysql.connector

conexion1=mysql.connector.connect(host="localhost",
                                 user="root",
                                 passwd="cuenca",
                                 database="chatBot")

def guardarInformacion(nombre, correo):
    cursor1=conexion1.cursor()
    sql="insert into registrodato(nombre, correo) values (%s,%s)"
    datos=(nombre, correo)
    cursor1.execute(sql, datos)
    conexion1.commit()
    ##conexion1.close()
    
##guardarInformacion("Nicolas", "nikoap77@gmail.com")


def Find(string): 
    regex = r"(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'\".,<>?«»“”‘’]))"
    url = re.findall(regex,string)     
    for x in url:
        return x[0]
    
TOKEN = "1389093712:AAElKiyc0Qgo0ahEE54uduPcFrq7rkSwZvg" #Cambialo por tu token
URL = "https://api.telegram.org/bot" + TOKEN + "/"

def update(offset):
    #Llamar al metodo getUpdates del bot, utilizando un offset
    respuesta = requests.get(URL + "getUpdates" + "?offset=" + str(offset))
    #Telegram devolvera todos los mensajes con id IGUAL o SUPERIOR al offset
 
 
    #Decodificar la respuesta recibida a formato UTF8
    mensajes_js = respuesta.content.decode("utf8")
 
    #Convertir el string de JSON a un diccionario de Python
    mensajes_diccionario = json.loads(mensajes_js)
 
    #Devolver este diccionario
    return mensajes_diccionario

def leer_mensaje(mensaje):
 
    #Extraer el texto, nombre de la persona e id del último mensaje recibido
    texto = mensaje["message"]["text"]
    persona = mensaje["message"]["from"]["first_name"]
    id_chat = mensaje["message"]["chat"]["id"]
 
    #Calcular el identificador unico del mensaje para calcular el offset
    id_update = mensaje["update_id"]
 
    #Devolver las dos id, el nombre y el texto del mensaje
    return id_chat, persona, texto, id_update

def sendMessageIBM(mensaje):
    aux=True
    res=''
    if("si " in mensaje or "no " in mensaje):
        lista = mensaje.split(" ")
        print("")
        print(lista)
        #send_message(rid,ejecutar(lista))
        aux=False
        return ejecutar(lista)
    if(" " in mensaje and "@" in mensaje):
        per= mensaje.split(" ")
        if(len(per)==2):
            #persona=Persona()
            guardarInformacion(per[0],per[1])
        else:
            res="Mal proceso de registro intente nuevamente"      

    verificar= Find(mensaje)
    if(verificar != None):
        response = service.analyze(
        url=verificar,
        features=Features(concepts=ConceptsOptions())).get_result()
        jsonData = json.dumps(response, indent=2)
        jsonToPython = json.loads(jsonData)
        #v = jsonToPython["output"]
        v = jsonToPython["concepts"]
        res1 = v[0]['text']
        print(res1)
        print(json.dumps(response, indent=2))
        aux=False
        return " Segun el analisis lo mas relevante de la pagina es: " + str(res1)
    
    print(mensaje)
    mensaje= traducir(mensaje)
    authenticator = IAMAuthenticator('wr9L0wufmqQYs55XaX5TMcJhmBfpEV-mbrqJHLlmUdZd')
    assistant = AssistantV2(
    version='2020-04-01',
    authenticator = authenticator
    )
    assistant.set_service_url('https://api.us-south.assistant.watson.cloud.ibm.com/instances/0efc759f-8634-41fc-b8cd-830039910c87')
    session = assistant.create_session(
    assistant_id='6418cb0f-f844-4814-bf1c-416be87e8b53'
    ).get_result()
    message = assistant.message(
    '6418cb0f-f844-4814-bf1c-416be87e8b53',
    session['session_id'],
    input={
        'message_type': 'text',
        'text': "'"+mensaje[0]+"'"
        #'text': "'"+mensaje+"'"
    }
    ).get_result()   
    jsonData = json.dumps(message, indent=2)
    jsonToPython = json.loads(jsonData)
    v = jsonToPython["output"]
    res = v['generic'][0]['text']
    return traducir2(res,mensaje[1])
    

    
    
def enviar_mensaje(idchat, texto):
    #Llamar el metodo sendMessage del bot, passando el texto y la id del chat
    requests.get(URL + "sendMessage?text=" + texto + "&chat_id=" + str(idchat))
    
    ##Variable para almacenar la ID del ultimo mensaje procesado
ultima_id = 0


 
while(True):
    mensajes_diccionario = update(ultima_id)
    for i in mensajes_diccionario["result"]:
 
        #Llamar a la funcion "leer_mensaje()"
        idchat, nombre, texto, id_update = leer_mensaje(i)
 
        #Si la ID del mensaje es mayor que el ultimo, se guarda la ID + 1
        if id_update > (ultima_id-1):
            ultima_id = id_update + 1
 
        #Generar una respuesta a partir de la informacion del mensaje
        #if "Hola" in texto:
            #texto_respuesta = "Hola, " + nombre + "!"
        #elif "Adios" in texto:
            #texto_respuesta = "Hasta pronto!"
        #else:
            #texto_respuesta = "Has escrito: \"" + texto + "\""
        try:
            texto_respuesta= sendMessageIBM(texto)
        except KeyError:
            texto_respuesta="No le entendido"
 
        #Enviar la respuesta
        enviar_mensaje(idchat, texto_respuesta)
 
    #Vaciar el diccionario
    mensajes_diccionario = []
